In [14]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, Sequence, MetaData
from sqlalchemy.orm import mapped_column, DeclarativeBase
from datetime import datetime

In [2]:
full_path=r'./data/PacificOcean.csv'

In [4]:
with open(full_path,'rb') as f:
    data=pd.read_csv(f,encoding='utf-8')
data

,code,location,city,country,lat,lon
0,waka,Wakkanai,稚内港-北海道稚内市,Japan,45.410000,141.690000
1,abas,Abashiri,网走港-北海道网走市,Japan,44.020000,144.290000
2,hana,Hanasaki,北海道根室市,Japan,43.280000,145.570000
3,kush,Kushiro,北海道钏路市,Japan,42.980000,144.370000
4,hako,Hakodate,北海道函馆市,Japan,41.780000,140.720000
...,...,...,...,...,...,...
168,stdpr,"Salinas, Puerto Rico",萨莱纳,PuertoRico,17.949063,-66.225967
169,aracS,Arecibo,阿雷西博,PuertoRico,18.480500,-66.702400
170,gtdpr,"Guayanilla, Puerto Rico",瓜亚尼亚港,PuertoRico,18.005219,-66.766697
171,maya,Mayaguez_PR,马亚圭斯,PuertoRico,18.217600,-67.158800


In [5]:
data.head()

,code,location,city,country,lat,lon
0,waka,Wakkanai,稚内港-北海道稚内市,Japan,45.41,141.69
1,abas,Abashiri,网走港-北海道网走市,Japan,44.02,144.29
2,hana,Hanasaki,北海道根室市,Japan,43.28,145.57
3,kush,Kushiro,北海道钏路市,Japan,42.98,144.37
4,hako,Hakodate,北海道函馆市,Japan,41.78,140.72


In [6]:
data.columns

Index(['code', 'location ', 'city', 'country', 'lat', 'lon'], dtype='object')

In [8]:
data.iloc[1]

code               abas
location       Abashiri
city         网走港-北海道网走市
country           Japan
lat               44.02
lon              144.29
Name: 1, dtype: object

In [11]:
# 数据库的配置，配置借鉴自 django 的 settings 的结构
DATABASES = {
    'default': {
        'ENGINE': 'mysqldb',  # 数据库引擎
        'NAME': 'surge_global_sys',  # 数据库名
        'USER': 'root',  # 账号
        'PASSWORD': '123456',
        'HOST': '127.0.0.1',  # HOST
        'POST': 3306,  # 端口
        'OPTIONS': {
            "init_command": "SET foreign_key_checks = 0;",
        },
    },
}

### 尝试将行政区域信息提取并写入 tb:region_info 

In [17]:
DEFAULT_COUNTRY_INDEX = -1

In [12]:
class DbFactory:
    """
        数据库工厂
    """

    def __init__(self, db_mapping: str = 'default', engine_str: str = None, host: str = None, port: str = None,
                 db_name: str = None,
                 user: str = None,
                 pwd: str = None):
        """
            mysql 数据库 构造函数
        :param db_mapping:
        :param engine_str:
        :param host:
        :param port:
        :param db_name:
        :param user:
        :param pwd:
        """
        db_options = DATABASES.get(db_mapping)
        self.engine_str = engine_str if engine_str else db_options.get('ENGINE')
        self.host = host if host else db_options.get('HOST')
        self.port = port if port else db_options.get('POST')
        self.db_name = db_name if db_name else db_options.get('NAME')
        self.user = user if user else db_options.get('USER')
        self.password = pwd if pwd else db_options.get('PASSWORD')
        # TypeError: Invalid argument(s) 'encoding' sent to create_engine(), using configuration MySQLDialect_mysqldb/QueuePool/Engine.  Please check that the keyword arguments are appropriate for this combination of components.
        self.engine = create_engine(
            f"mysql+{self.engine_str}://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}",
            pool_pre_ping=True, future=True)
        self._session_def = sessionmaker(bind=self.engine)

    @property
    def Session(self) -> sessionmaker:
        if self._session_def is None:
            self._session_def = sessionmaker(bind=self.engine)
        return self._session_def()

In [15]:
class BaseMeta(DeclarativeBase):
    pass


class IIdModel(BaseMeta):
    __abstract__ = True
    id: Mapped[int] = mapped_column(primary_key=True)


class IDel(BaseMeta):
    """
        软删除 抽象父类
    """
    __abstract__ = True
    is_del: Mapped[int] = mapped_column(nullable=False, default=0)


class IModel(BaseMeta):
    """
        model 抽象父类，主要包含 创建及修改时间
    """
    __abstract__ = True
    gmt_create_time: Mapped[datetime] = mapped_column(default=datetime.utcnow)
    gmt_modify_time: Mapped[datetime] = mapped_column(default=datetime.utcnow)


In [18]:
class RegionInfo(IIdModel, IDel, IModel):
    """
        行政区划表
    """
    location: Mapped[str] = mapped_column()
    city: Mapped[str] = mapped_column()
    city_name_ch: Mapped[str] = mapped_column()
    country: Mapped[int] = mapped_column(nullable=False, default=DEFAULT_COUNTRY_INDEX)
    __tablename__ = 'region_info'

In [22]:
num_columns=data.shape[1]
num_columns

6

In [23]:
num_rows=data.shape[0]
num_rows

173

In [24]:
data.iloc[0]

code               waka
location       Wakkanai
city         稚内港-北海道稚内市
country           Japan
lat               45.41
lon              141.69
Name: 0, dtype: object

In [29]:
type(data.iloc[0])

pandas.core.series.Series

In [34]:
data.iloc[0].loc['lat']

45.41

In [39]:
set(data['country'].tolist())

{'Australia',
 'CostaRica',
 'ElSalvador',
 'Fiji Islands',
 'France',
 'IndependentStateofWesternSamoa',
 'Japan',
 'Mexico',
 'NewZealand',
 'Nicaragua',
 'PapuaNewGuinea',
 'Philippines',
 'PuertoRico',
 'SolomonIslands',
 'SouthernKorea',
 'U.S.A.Samoa',
 'USA',
 'Vanuatu',
 'Viet Nam'}

In [ ]:
for index in range(num_rows):
    series_column=data.iloc[index]
    region_model=RegionInfo(location=series_column.loc['location'],city=series_column.loc['location'],city_name_ch='',)